In [1]:
import kabuki
import os
import hddm
import matplotlib.pyplot as plt
import pickle
from joblib import Parallel,delayed
from glob import glob
from kabuki.analyze import gelman_rubin
import arviz
import numpy as np
from patsy import dmatrix 


from sklearn.preprocessing import StandardScaler

/opt/conda/lib/python3.8/site-packages/IPython/parallel.py:12: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  warn("The `IPython.parallel` package has been deprecated since IPython 4.0. "


In [ ]:
# load data
data = hddm.load_csv('./data.csv')

# stimulus onset time is 200, we should add it.
data['rt']=data['rt']+0.2
data.loc[(data['response']==1)&(data['stimulus']=='face'),'_response'] = 0 #face
data.loc[(data['response']==1)&(data['stimulus']=='car'),'_response'] = 1 #car
data.loc[(data['response']==0)&(data['stimulus']=='face'),'_response'] = 1
data.loc[(data['response']==0)&(data['stimulus']=='car'),'_response'] = 0
# rt of correct time should be positive, and rt of incorrect time should be negative.
df = data
df['response'] = df['_response']
df = hddm.utils.flip_errors(df)

# normalise the cpp signal
df['cpp_slope']=StandardScaler().fit_transform(df['cpp_slope'].to_numpy().reshape(-1,1))
df['cpp_amplitude']=StandardScaler().fit_transform(df['cpp_amplitude'].to_numpy().reshape(-1,1))
df['cpp_peak']=StandardScaler().fit_transform(df['cpp_peak'].to_numpy().reshape(-1,1))

In [4]:
# link function for stimulus, one stimulus should be coded 0, another should be coded 1
def z_link_func(x, data=data):
        stim = (np.asarray(dmatrix('0 + C(s, [[0], [1]])',
                                  {'s': data.stimulus.loc[x.index]},return_type='dataframe'))
        )
        # Apply z = (1 - x) to flip them along 0.5
        z_flip = np.subtract(stim, x.to_frame())
        # The above inverts those values we do not want to flip,
        # so invert them back
        z_flip[stim == 0] *= -1
        return z_flip


NameError: name 'data' is not defined

In [3]:
def v_link_func(x, data=data):
    stim = (np.asarray(dmatrix('0 + C(s,[[1],[-1]])', {'s':data.stimulus.ioc[x.index]})))
    return x * stim

NameError: name 'data' is not defined

In [ ]:
# plot for checking the rt distribution of two response
fig = plt.figure()

ax = fig.add_subplot(111, xlabel='RT', ylabel='count', title='RT distributions')

for i, subj_data in df.groupby('subj_idx'):

    subj_data.rt.hist(bins=20, histtype='step', ax=ax)